##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 単一ワーカーでのマルチ GPU トレーニングの移行

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/mirrored_strategy"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

このガイドでは、単一ワーカーのマルチ GPU ワークフローを TensorFlow 1 から TensorFlow 2 に移行する方法を実演します。

1 台のマシンのマルチ GPU で同期トレーニングを実行するには、

- TensorFlow 1 では、`tf.distribute.MirroredStrategy` で `tf.estimator.Estimator` API を使用します。
- TensorFlow 2 では、[Keras Model.fit](https://www.tensorflow.org/tutorials/distribute/keras) または、[カスタムトレーニングループ](https://www.tensorflow.org/tutorials/distribute/custom_training)を `tf.distribute.MirroredStrategy` で使用できます。詳しくは、[TensorFlow を使用した分散トレーニング](https://www.tensorflow.org/guide/distributed_training#mirroredstrategy)ガイドを参照してください。

## セットアップ

まず、インポートとデモ用の単純なデータセットから始めます。

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: tf.estimator.Estimator を使った単一ワーカーでの分散トレーニング

この例では、TensorFlow 1 での単一ワーカーのマルチ GPU トレーニングの標準ワークフローを示します。`tf.estimator.Estimator` の `config` パラメータを使用して、分散ストラテジー （`tf.distribute.MirroredStrategy`）を設定する必要があります。

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

strategy = tf1.distribute.MirroredStrategy()
config = tf1.estimator.RunConfig(
    train_distribute=strategy, eval_distribute=strategy)
estimator = tf1.estimator.Estimator(model_fn=_model_fn, config=config)

train_spec = tf1.estimator.TrainSpec(input_fn=_input_fn)
eval_spec = tf1.estimator.EvalSpec(input_fn=_eval_input_fn)
tf1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## TensorFlow 2: Keras を使った単一ワーカーでのトレーニング

TensorFlow 2 に移行する場合は、`tf.distribute.MirroredStrategy` で Keras API を使用できます。

モデル構築に `tf.keras` API を使用し、トレーニングに Keras `Model.fit` を使用する場合、主な違いは、`tf.estimator.Estimator` の `config` を定義する代わりに Keras モデル、オプティマイザ、および指標を`Strategy.scope` のコンテキストでインスタンス化することです。

カスタムトレーニングループを使用する必要がある場合は、[カスタムトレーニングループで tf.distribute.Strategy を使用する](https://www.tensorflow.org/guide/distributed_training#using_tfdistributestrategy_with_custom_training_loops)ガイドを確認してください。

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss='mse')
model.fit(dataset)
model.evaluate(eval_dataset, return_dict=True)

## 次のステップ

TensorFlow 2 の `tf.distribute.MirroredStrategy` を使った分散トレーニングの詳細については、次のドキュメントを参照してください。

- [Keras を使った単一マシンでの分散トレーニング](../../tutorials/distribute/keras)チュートリアル
- [カスタムトレーニングループを使った単一マシンでの分散トレーニング](../../tutorials/distribute/custom_training)チュートリアル
- [TensorFlow を使った分散トレーニング](../../guide/distributed_training)ガイド
- [マルチ GPU の使用](../../guide/gpu#using_multiple_gpus)ガイド
- [マルチ GPU 単一ホストのパフォーマンスを最適化する（TensorFlow Profiler を使用）](../../guide/gpu_performance_analysis#2_optimize_the_performance_on_the_multi-gpu_single_host)ガイド